In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('MinMax_Scaler_result.csv',encoding='cp949')

In [3]:

from sklearn.model_selection import train_test_split

In [4]:
a = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [5]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT',  'KOSPI', '환율', 'GDP', 
       '국고채', '유가등락률',
       'KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]


In [6]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 34) (11736, 34) (27381,) (11736,)


In [7]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [9]:
for i in keys_list:
    model = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=1)
    model.fit(data_sets[i][0], data_sets[i][1])
                            
                    # 결정계수
    relation_square = model.score(data_sets[i][0], data_sets[i][1])
    print('결정계수 : ', relation_square)
                    
                    # 예측
    pred_y = model.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    matrix_kd_tree_uniform_BOST.append((i,pr,r,f1,rca,model.score(test_x, test_y),cfm))
                # 정확도 확인
    print(i)
    print('정확도 : ', metrics.accuracy_score(test_y, pred_y))
    print(classification_report(test_y, pred_y, target_names=['사고보증', '정상보증']))
    print('\n')

결정계수 :  0.9993297886600241
enn
정확도 :  0.9234832992501704
              precision    recall  f1-score   support

        사고보증       0.93      0.99      0.96     10666
        정상보증       0.75      0.24      0.36      1070

    accuracy                           0.92     11736
   macro avg       0.84      0.62      0.66     11736
weighted avg       0.91      0.92      0.91     11736



결정계수 :  0.998931860779137
ada
정확도 :  0.901414451261077
              precision    recall  f1-score   support

        사고보증       0.93      0.96      0.95     10666
        정상보증       0.45      0.33      0.38      1070

    accuracy                           0.90     11736
   macro avg       0.69      0.65      0.66     11736
weighted avg       0.89      0.90      0.89     11736



결정계수 :  1.0
smte
정확도 :  0.8352079072937969
              precision    recall  f1-score   support

        사고보증       0.95      0.87      0.91     10666
        정상보증       0.28      0.50      0.35      1070

    accuracy           

In [10]:
matrix_kd_tree_uniform_BOST

[('enn',
  0.75,
  0.2411214953271028,
  0.3649222065063649,
  0.6165292456946784,
  0.9234832992501704,
  array([[10580,    86],
         [  812,   258]], dtype=int64)),
 ('ada',
  0.4455569461827284,
  0.33271028037383177,
  0.38095238095238093,
  0.6455882172542325,
  0.901414451261077,
  array([[10223,   443],
         [  714,   356]], dtype=int64)),
 ('smte',
  0.27593360995850624,
  0.497196261682243,
  0.3549032688458973,
  0.6831565407417403,
  0.8352079072937969,
  array([[9270, 1396],
         [ 538,  532]], dtype=int64)),
 ('smtt',
  0.4460338101430429,
  0.3205607476635514,
  0.3730288200108755,
  0.6403103757068929,
  0.9017552828902522,
  array([[10240,   426],
         [  727,   343]], dtype=int64)),
 ('tk',
  0.9007936507936508,
  0.21214953271028036,
  0.34341906202723144,
  0.6049028181083749,
  0.9260395364689843,
  array([[10641,    25],
         [  843,   227]], dtype=int64)),
 ('sm',
  0.4890190336749634,
  0.3121495327102804,
  0.38106103822019394,
  0.6397143688